# 딥 러닝을 이용한 자연어 처리 입문

아래 링크의 E-book을 보고 실습한 내용입니다.

WikiDocs 주소: https://wikidocs.net/31766

# 6장 토픽 모델링

## 2절 잠재 디리클레 할당 (Latent Dirichlet Allocation, LDA)


## 뉴스 그룹 데이터를 활용한 LDA 실습 - gensim

### 텍스트 전처리


In [1]:
import pandas as pd
from sklearn.datasets import fetch_20newsgroups
from nltk.corpus import stopwords

dataset = fetch_20newsgroups(
    shuffle=True, random_state=1, remove=('headers', 'footers', 'quotes'))
documents = dataset.data


news_df = pd.DataFrame({"document": documents})
# 특수 문자 제거
news_df['clean_doc'] = news_df['document'].str.replace(
    r"[^a-zA-Z]", " ", regex=True)
# 길이가 짧은 단어 제거 및 단어 소문자 변환
news_df['clean_doc'] = news_df['clean_doc'].apply(
    lambda x: ' '.join([w.lower() for w in x.split() if len(w) > 3]))

# 불용어 제거
stop_words = stopwords.words('english')
tokenized_doc = news_df['clean_doc'].apply(lambda x: x.split())
tokenized_doc = tokenized_doc.apply(
    lambda x: [item for item in x if item not in stop_words])

tokenized_doc


0        [well, sure, story, seem, biased, disagree, st...
1        [yeah, expect, people, read, actually, accept,...
2        [although, realize, principle, strongest, poin...
3        [notwithstanding, legitimate, fuss, proposal, ...
4        [well, change, scoring, playoff, pool, unfortu...
                               ...                        
11309    [danny, rubenstein, israeli, journalist, speak...
11310                                                   []
11311    [agree, home, runs, clemens, always, memorable...
11312    [used, deskjet, orange, micros, grappler, syst...
11313    [argument, murphy, scared, hell, came, last, y...
Name: clean_doc, Length: 11314, dtype: object

In [2]:
from gensim import corpora


# Vocabulary 생성
dictionary = corpora.Dictionary(tokenized_doc)
# 정수 인코딩 및 빈도수 기록
corpus = [dictionary.doc2bow(text) for text in tokenized_doc]

print("(word_id, 빈도수), 뉴스 2의 결과:", corpus[1])

(word_id, 빈도수), 뉴스 2의 결과: [(52, 1), (55, 1), (56, 1), (57, 1), (58, 1), (59, 1), (60, 1), (61, 1), (62, 1), (63, 1), (64, 1), (65, 1), (66, 2), (67, 1), (68, 1), (69, 1), (70, 1), (71, 2), (72, 1), (73, 1), (74, 1), (75, 1), (76, 1), (77, 1), (78, 2), (79, 1), (80, 1), (81, 1), (82, 1), (83, 1), (84, 1), (85, 2), (86, 1), (87, 1), (88, 1), (89, 1)]


In [3]:
print("word_id가 66인 단어를 dictionary에서 찾기:", dictionary[66])
print("전체 단어:", len(dictionary))

word_id가 66인 단어를 dictionary에서 찾기: faith
전체 단어: 64281


In [4]:
import gensim

# LDA 학습을 위해 토픽의 개수를 지정
NUM_TOPICS = 20

# LDA 모델 객체 생성
# - passes: 알고리즘 동작 횟수, 적절히 수렴할 수 있도록 적당한 값 할당
ldamodel = gensim.models.ldamodel.LdaModel(
    corpus, num_topics=NUM_TOPICS, id2word=dictionary, passes=15)

# 토픽 목록 출력
# - num_words: 총 몇 개의 단어를 출력할 것인지
topics = ldamodel.print_topics(num_words=4)
print("각 토픽별 단어 분포")
print("(토픽ID, 중요도*단어, ... 중요도*단어")
for topic in topics:
    print(topic)


각 토픽별 단어 분포
(토픽ID, 중요도*단어, ... 중요도*단어
(0, '0.011*"armenian" + 0.010*"israel" + 0.010*"people" + 0.010*"armenians"')
(1, '0.028*"jesus" + 0.016*"christian" + 0.016*"bible" + 0.014*"church"')
(2, '0.023*"wire" + 0.023*"ground" + 0.019*"water" + 0.015*"nuclear"')
(3, '0.011*"monitor" + 0.008*"video" + 0.008*"cable" + 0.007*"card"')
(4, '0.016*"would" + 0.015*"thanks" + 0.014*"know" + 0.013*"like"')
(5, '0.036*"window" + 0.020*"motif" + 0.018*"widget" + 0.013*"application"')
(6, '0.012*"maine" + 0.008*"latin" + 0.008*"erzurum" + 0.006*"jonathan"')
(7, '0.017*"would" + 0.012*"people" + 0.010*"think" + 0.007*"like"')
(8, '0.041*"keyboard" + 0.007*"sequences" + 0.007*"echo" + 0.006*"kong"')
(9, '0.020*"file" + 0.013*"program" + 0.010*"available" + 0.009*"files"')
(10, '0.010*"good" + 0.009*"bike" + 0.006*"cars" + 0.006*"engine"')
(11, '0.013*"lost" + 0.012*"colorado" + 0.009*"houston" + 0.008*"francis"')
(12, '0.009*"said" + 0.009*"would" + 0.009*"time" + 0.008*"like"')
(13, '0.025*"tobacco" 

### pyLDAvis 결과 해설

- 각 원은 토픽들을 나타낸다. (20개)
- 원과 원 사이의 거리는 각 토픽들이 얼마나 다른지 보여준다. (가까우면 유사한 토픽)

- 주의
  - LDA 결과에서는 토픽번호가 0번부터 인덱싱이 시작
  - LDA 시각화 결과에서는 토픽번호가 1번부터 인덱싱이 시작


In [5]:
import pyLDAvis.gensim_models

pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(ldamodel, corpus, dictionary)
pyLDAvis.display(vis)


/usr/local/lib/python3.8/dist-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(
/usr/local/lib/python3.8/dist-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/usr/local/lib/python3.8/dist-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/usr/local/lib/python3.8/dist-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/usr/local/lib/python3.8/dist-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is dep

In [6]:
print("문서별 토픽 분포")
for i, topic_list in enumerate(ldamodel[corpus]):
    if i==5:
        break
    print(f'{i}번째 문서의 토픽 분포:{topic_list}')

문서별 토픽 분포
0번째 문서의 토픽 분포:[(0, 0.63176805), (7, 0.3330463), (19, 0.021206401)]
1번째 문서의 토픽 분포:[(0, 0.06009686), (1, 0.040245783), (7, 0.73231006), (8, 0.055041105), (9, 0.066329174), (18, 0.027535109)]
2번째 문서의 토픽 분포:[(0, 0.31876937), (4, 0.19725855), (7, 0.46999764)]
3번째 문서의 토픽 분포:[(3, 0.01859758), (4, 0.2513879), (7, 0.22811705), (12, 0.08558476), (16, 0.33720285), (19, 0.06813194)]
4번째 문서의 토픽 분포:[(9, 0.091015205), (12, 0.55512583), (17, 0.32236418)]


In [7]:
# Dataframe으로 보기 쉽게 정리
df = pd.DataFrame()
for i, topic_list in enumerate(ldamodel[corpus]):
    doc = topic_list[0] if ldamodel.per_word_topics else topic_list
    doc = sorted(doc, key=lambda x: (x[1]), reverse=True)
    for j, (topic_id, prop_topic) in enumerate(doc):
        if j == 0:
            df = df.append(pd.Series([int(topic_id), round(
                prop_topic, 4), topic_list]), ignore_index=True)
        else:
            break


In [8]:
topic_table = df
topic_table.columns = ["1등 토픽", "비중", "나머지 토픽들의 비중"]
topic_table["1등 토픽"] = topic_table["1등 토픽"].astype(int)
topic_table[:10]


,1등 토픽,비중,나머지 토픽들의 비중
0,0,0.6317,"[(0, 0.6317219), (7, 0.33308017), (19, 0.02121..."
1,7,0.7323,"[(0, 0.060097262), (1, 0.040242594), (7, 0.732..."
2,7,0.4700,"[(0, 0.31877655), (4, 0.19725081), (7, 0.46999..."
3,16,0.3372,"[(3, 0.018597592), (4, 0.25131094), (7, 0.2280..."
4,12,0.5552,"[(9, 0.09094425), (12, 0.5551676), (17, 0.3223..."
5,7,0.5844,"[(1, 0.2596395), (5, 0.06439281), (7, 0.584434..."
6,7,0.4345,"[(3, 0.20723943), (4, 0.0969185), (5, 0.014466..."
7,7,0.6133,"[(0, 0.22961026), (7, 0.61326337), (16, 0.1431..."
8,7,0.4954,"[(1, 0.2912268), (7, 0.49543366), (8, 0.060244..."
9,10,0.4722,"[(0, 0.017198863), (4, 0.20684989), (7, 0.2926..."
